# Ventilator Pressure Prediction

This notebook trains an LSTM model to predict ventilator pressure based on time-series data.

<img src="https://raw.githubusercontent.com/google/deluca-lung/main/assets/2020-10-02%20Ventilator%20diagram.svg" alt="Ventilator Diagram">

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load datasets
train_set = pd.read_csv('../data/train.csv')
test_set = pd.read_csv('../data/test.csv')

# Check for missing values
print(train_set.isna().sum())

In [ ]:
# Preprocess data
def preprocess_data(df, is_train=True):
    features = ['R', 'C', 'time_step', 'u_in', 'u_out']
    if is_train:
        features.append('pressure')
    
    # Group by breath_id and ensure 80 time steps
    grouped = df.groupby('breath_id').apply(lambda x: x[features].head(80)).reset_index()
    
    X = grouped[features[:-1] if is_train else features].values.reshape(-1, 80, len(features) - (1 if is_train else 0))
    X = X.astype(np.float32)
    
    # Normalize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
    
    if is_train:
        y = grouped['pressure'].values.reshape(-1, 80, 1)
        return X, y, scaler
    return X, scaler

X_train, y_train, scaler = preprocess_data(train_set)
X_test, _ = preprocess_data(test_set, is_train=False)

In [ ]:
# Build LSTM model
model = Sequential([
    LSTM(128, input_shape=(80, 5), return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mae', metrics=['mae'])
model.summary()

In [ ]:
# Train model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=1)

# Save model
model.save('../models/ventilator_lstm.h5')

In [ ]:
# Evaluate on test set
predictions = model.predict(X_test)
print(predictions.shape)